In [14]:
#pip install psycopg2
#pip install pyodbc
#pip install csv

import psycopg2
import pyodbc
import csv

# Kết nối đến PostgreSQL
postgres_conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="test_aptech_4",
    user="postgres",
    password="admin123"
)

# Kết nối đến SQL Server
sql_server_conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=192.168.100.27;"
    "Database=test_pg_data;"
    "UID=root;"
    "PWD=admin123;"
)

# Lấy danh sách các bảng trong PostgreSQL
postgres_cursor = postgres_conn.cursor()
postgres_cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
postgres_tables = postgres_cursor.fetchall()

# Lấy danh sách các bảng trong SQL Server
sql_server_cursor = sql_server_conn.cursor()
sql_server_cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE';")
sql_server_tables = sql_server_cursor.fetchall()

In [15]:
# so sánh cấu trúc bảng của 2 hệ cơ sở dữ liệu
structs_list = []

# So sánh dữ liệu trong các bảng
for postgres_table in postgres_tables:
    postgres_table_name = postgres_table[0]
    sql_server_table_name = None

    # Tìm bảng tương ứng trong SQL Server
    for sql_server_table in sql_server_tables:
        if sql_server_table[0].lower() == postgres_table_name.lower():
            sql_server_table_name = sql_server_table[0]
            break
    
    if sql_server_table_name:
        # Lấy danh sách các cột trong PostgreSQL
        postgres_cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{postgres_table_name}' ORDER BY column_name DESC;")
        postgres_columns = postgres_cursor.fetchall()

        # Lấy danh sách các cột trong SQL Server
        sql_server_cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{sql_server_table_name}' ORDER BY COLUMN_NAME DESC;")
        sql_server_columns = sql_server_cursor.fetchall()

        for postgres_column, sql_server_column in zip(postgres_columns, sql_server_columns):
            postgres_column_name = postgres_column[0]
            sql_server_column_name = sql_server_column[0]

            postgres_data_type = None
            sql_data_type = None
            #kiểm tra xem kiểu của cột dữ liệu có trống ko thì lấy kiểu dữ liệu của cột
            if len(postgres_column) > 1:
                postgres_data_type = postgres_column[1]
            if len(sql_server_column) > 1:
                sql_data_type = sql_server_column[1]
            structs_list.append({
                    "table_name": postgres_table_name,
                    "column_name": postgres_column_name,
                    "postgres_type": postgres_data_type,
                    "sql_server_type": sql_data_type
                })
    # Xuất dữ liệu khác nhau ra file CSV
    if structs_list:
        with open("different_struct.csv", "w", newline="") as csvfile:
            fieldnames = ["table_name", "column_name", "postgres_type", "sql_server_type"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for data in structs_list:
                writer.writerow(data)
        print("Đã xuất dữ liệu ra file different_struct.csv")
    else:
        print("Không có dữ liệu để xuất ra file CSV.")

    # # Đóng kết nối
    # postgres_cursor.close()
    # postgres_conn.close()
    # sql_server_cursor.close()
    # sql_server_conn.close()


Không có dữ liệu khác nhau để xuất ra file CSV.
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv
Đã xuất dữ liệu khác nhau ra file different_struct.csv


In [16]:
# Tìm Dữ liệu khác nhau trong các bảng
different_data = []

# So sánh dữ liệu trong các bảng
for postgres_table in postgres_tables:
    postgres_table_name = postgres_table[0]
    sql_server_table_name = None

    # Tìm bảng tương ứng trong SQL Server
    for sql_server_table in sql_server_tables:
        if sql_server_table[0].lower() == postgres_table_name.lower():
            sql_server_table_name = sql_server_table[0]
            break

    if sql_server_table_name:
        # Lấy danh sách các cột trong PostgreSQL
        postgres_cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{postgres_table_name}' ORDER BY column_name DESC;")
        postgres_columns = postgres_cursor.fetchall()

        # Lấy danh sách các cột trong SQL Server
        sql_server_cursor.execute(f"SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{sql_server_table_name}' ORDER BY COLUMN_NAME DESC;")
        sql_server_columns = sql_server_cursor.fetchall()

        # Trim dữ liệu trong các cột
        for postgres_column, sql_server_column in zip(postgres_columns, sql_server_columns):
            postgres_column_name = postgres_column[0]
            sql_server_column_name = sql_server_column[0]

            postgres_data_type = None
            sql_data_type = None
            #kiểm tra xem kiểu của cột dữ liệu có trống ko thì lấy kiểu dữ liệu của cột
            if len(postgres_column) > 1:
                postgres_data_type = postgres_column[1]
            if len(sql_server_column) > 1:
                sql_data_type = sql_server_column[1]
            # postgres_data_type = 'null'
            pg_type = ['smallint', 'integer', 'bytea', 'date', 'real']
            print(postgres_data_type)
            if postgres_data_type in pg_type and postgres_column_name is not None:
                # Lấy dữ liệu từ PostgreSQL
                postgres_select_query = f"SELECT ({postgres_column_name}) FROM {postgres_table_name};"
            else:
                if postgres_column_name is not None:
                    postgres_select_query = f"SELECT TRIM({postgres_column_name}) FROM {postgres_table_name};"
            postgres_cursor.execute(postgres_select_query)
            postgres_data = postgres_cursor.fetchall()

            sql_type = ['ntext', 'image']
            if sql_data_type in sql_type and sql_server_column_name is not None:
                # Lấy dữ liệu từ SQL Server
                sql_server_select_query = f"SELECT ({sql_server_column_name}) FROM {sql_server_table_name};"
            else:
                if sql_server_column_name is not None:
                    sql_server_select_query = f"SELECT RTRIM(LTRIM({sql_server_column_name})) FROM {sql_server_table_name};"
            sql_server_cursor.execute(sql_server_select_query)
            sql_server_data = sql_server_cursor.fetchall()

            # So sánh dữ liệu
            if postgres_data != sql_server_data:
                different_data.append({
                    "table_name": postgres_table_name,
                    "column_name": postgres_column_name,
                    "postgres_data": postgres_data,
                    "sql_server_data": sql_server_data
                })

# Xuất dữ liệu khác nhau ra file CSV
if different_data:
    with open("different_data.csv", "w", newline="") as csvfile:
        fieldnames = ["table_name", "column_name", "postgres_data", "sql_server_data"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in different_data:
            writer.writerow(data)
    print("Đã xuất dữ liệu khác nhau ra file different_data.csv")
else:
    print("Không có dữ liệu khác nhau để xuất ra file CSV.")

# Đóng kết nối
postgres_cursor.close()
postgres_conn.close()
sql_server_cursor.close()
sql_server_conn.close()


bytea
text
character varying
smallint
smallint
character
smallint
character varying
character varying
character varying
text
character varying
character varying
character varying
character varying
character varying
character varying
character varying
character varying
character varying
smallint
character varying
character varying
character varying
bytea
text
character varying
character varying
date
character varying
character varying
smallint
character varying
character varying
date
character varying
character varying
character varying
character varying
character varying
character
character varying
character varying
character varying
character varying
character varying
character varying
smallint
smallint
real
smallint
smallint
character varying
character varying
smallint
integer
smallint
smallint
character varying
character varying
date
smallint
character varying
character varying
character varying
character varying
character varying
character varying
date
smallint
date
real
smallint
c